# Data Collection & Cleaning of Ridership Project

### Importing Necessary Libraries & Modules

In [1]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode

/Users/deepthikurup/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/deepthikurup/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Cleaning Overview

For this project we decided to use a number of datasets:

1. Monthly Chicago Transit Authority (CTA) L (Metro System) Ridership data from the City of Chicago Data Portal
2. 2000, 2010 & 2020 Population Data from the US Census Bureau for each Zip Code in Cook County, IL. Cook County includes includes Chicago and a number of suburbs in the Chicago Metro Area. 
3. CTA dataset containing descriptive information about various stations in the CTA L system

Our goal was to combine all these datasets together, extract the relevant information, and analyze the information to get a better understanding of how population trends and ridership trends have changed over time and may be related with each other. This notebook includes the entire data collection, cleaning, and merging process that was developed as part of this project, which was consolidated into 9 functions. 

## Primary Dataset Cleaning

The first dataset we explored contained CTA L Ridership Data. This was downloaded via CSV from the City of Chicago Data Portal. This dataset contains, for each L Stop, the number of entries into the station aggregated at a monthly level from January 1st, 2001 to February 28th, 2024. The dataset we downloaded has nearly 40,000 records, and by looking at the head below, we can see all of the columns available


`Convert to code block to see what the raw dataframe looks like`

`dataset1 = pd.read_csv("siads_593_milestone1/src/data/CTARidershipMonthly_20240521.csv")`
`display(dataset1.head())`

We are looking to keep all the columns that contain ridership data, descriptive data, or data that can be used to merge this dataframe with the other datasets. To clean this dataset, we created the below function, `primary_dataset_cleaning`, in order to convert this CSV into a dataframe and keep relevant information. Eventually, we want to link this function to the City of Chicago Dataportal API, so we build this function in a way that can accomodate that with minimal changes. 

All of the adjustments made to columns, and column drops were done to ensure they met the criteria outlined above.

In [6]:
def primary_dataset_cleaning():
    
    # Converting CSV into dataframe
    monthly_ridership_df = pd.read_csv("../data/CTARidershipMonthly_20240521.csv")
    
    # Change the column names to Upper Case to make it easier to clean
    monthly_ridership_df_column = monthly_ridership_df.columns
    monthly_ridership_df.columns = monthly_ridership_df_column.str.upper()
    
    # Rename STATIONAME to STATION_NAME to increase readability and ease of merging with other datasets
    monthly_ridership_df.rename(columns = {'STATIONAME': 'STATION_NAME'}, inplace = True)

    # Drop Station_ID column as it doesn't have relevant information for this project
    monthly_ridership_df = monthly_ridership_df.drop(['STATION_ID'], axis = 1)

    return monthly_ridership_df

In [8]:
df = primary_dataset_cleaning()
df.head()

,STATION_NAME,MONTH_BEGINNING,AVG_WEEKDAY_RIDES,AVG_SATURDAY_RIDES,AVG_SUNDAY-HOLIDAY_RIDES,MONTHTOTAL
0,Howard,01/01/2001,6233.9,3814.5,2408.6,164447
1,Jarvis,01/01/2001,1489.1,1054.0,718.0,40567
2,Morse,01/01/2001,4412.5,3064.5,2087.8,119772
3,Loyola,01/01/2001,4664.5,3156.0,1952.8,125008
4,Granville,01/01/2001,3109.8,2126.0,1453.8,84189


`Convert to code block to see what the Ridership dataframe looks like after cleaning`
`display(primary_dataset_cleaning().head())`

Now that our primary dataset including ridership information has been cleaned, we want to clean our next dataset, which is Cook County population by Zip Code from the Decennial US Census. 

While all the data is from the same source, the US Census Bureau, the data was pulled from 3 different datasets. 2010 & 2020 data was from the same type of report, but for different Census', but that report wasn't available for 2000, therefore it had to be pulled from a different report that was formatted differently. The function below, `secondary_dataset_cleaning` takes all three datasets in CSV format, converts them into dataframes, and merges them together in a consistent format. 

`Convert to code block to see what the raw dataframes look like`

`dataset2_2000 = pd.read_csv("siads_593_milestone1/src/data/DECENNIALSF12000.H002-2024-05-27T193225.csv")`
`dataset2_2010 = pd.read_csv("siads_593_milestone1/src/data/DECENNIALSF12010.P1-Data.csv")`
`dataset2_2020 = pd.read_csv("siads_593_milestone1/src/data/DECENNIALDHC2020.P1-Data.csv")`

`display(dataset2_2000.head())`
`display(dataset2_2010.head())`
`display(dataset2_2020.head())`

## Secondary Dataset Cleaning

In [14]:
def secondary_dataset_cleaning():
    
    # Create Dataframes for each years Census Data
    pop_2000 = pd.read_csv("../data/DECENNIALSF12000.H002-2024-05-27T193225.csv")
    pop_2010 = pd.read_csv("../data/DECENNIALSF12010.P1-Data.csv")
    pop_2020 = pd.read_csv("../data/DECENNIALDHC2020.P1-Data.csv")

    # Begin cleaning of 2000 DataFrame   
    # 2000 Dataset does not match 2010 & 2020 Datsets. Manipulating pop_2000 to match 2010/2020 format

    # Index 0 has secondary headers that are not relevant for the purpose of this project
    pop_2000_cleaning = pop_2000.drop(pop_2000.index[1:], axis = 0)
    
    # Transposing to change zip codes from column headers to row headers
    pop_2000_cleaning = pop_2000_cleaning.transpose()

    # We want the Zip Codes to be in a column itself, rather than as the dataframes index
    pop_2000_cleaned = pop_2000_cleaning.reset_index().drop(0, axis = 0)
    
    # Renaming columns to make it easier to merge with the other dataframes
    pop_2000_cleaned.columns = ['NAME', 'P2000']

    # Removing commas from population numbers since 2010/2020 data does not have them
    pop_2000_cleaned['P2000'] = pop_2000_cleaned['P2000'].str.replace(',','')

    # End cleaning of 2000 Dataframe


    # Begin cleaning of 2010 & 2020 DataFrames

    #Removing unneeded columns
    pop_2010_cleaned = pop_2010.drop(0, axis = 0).drop(columns = ['GEO_ID', 'Unnamed: 3'])
    pop_2020_cleaned = pop_2020.drop(0, axis = 0).drop(columns = ['GEO_ID', 'Unnamed: 3'])

    # End cleaning of 2010 & 2020 DataFrames

    # Merging all three dataframes together and renaming columns to be more usable
    total_pop = (pop_2000_cleaned.merge(pop_2010_cleaned, on = "NAME", how = 'outer')
        .merge(pop_2020_cleaned, on = 'NAME', how = 'outer')
    )
    total_pop.columns = ['ZIP', '2000 Population', '2010 Population', '2020 Population']

    # Removing "ZCTA 5" to keep just the Zip Code
    total_pop['ZIP'] = total_pop['ZIP'].str.replace('ZCTA5 ', '')

    columns = ['2000 Population', '2010 Population', '2020 Population']

    total_pop[columns] = total_pop[columns].apply(lambda col: pd.to_numeric(col, errors = 'coerce'))

    total_pop = total_pop.dropna(subset = ['2000 Population', '2010 Population'], how = 'all')
    
    # Call remove_nan function to remove nan values from total_pop
    total_pop_without_nan = remove_nan(total_pop)

    return total_pop_without_nan

`Convert to code block to see what the population dataframe looks like after cleaning`
`display(secondary_dataset_cleaning().head())`

### Removing NaN Values from Secondary Dataset

After cleaning our population datasets, we noticed there are a number of `NaN` values. Some of them are due to the structure of the 2000 Dataset. To delinate between Zip Code Groups, the Census Bureau added dummy values. For example, when the Zip Codes starting with 605 (e.g. 60545) end, they add a dummy value of 605HH before starting the ZIP codes that start with 606. The rows with these dummy ZIP codes had NaN values for all three rows. Those rows were just dropped within secondary_dataset_cleaning itself. 

For the rest of the `NaN` values, the function `remove_nan` was created, which is called within the `secondary_dataset_cleaning` function. For most of the other rows that contained `NaN` values, data wasn't available for either 2000 or 2020. Using the change between two consecutive Census', we extrapolated the population to replace `NaN` values using absoluate nominal change. If the 2000 value for a row was `NaN`, we reversed the change from 2010 to 2020. If the 2020 value was missing, we carried forward the change from 2000 to 2010 to replace the `NaN` values. There were a few rows that had `NaN` values in two of the columns. When those ZIP Codes were observed on a map, we saw that those ZIP codes were far from the City of Chicago and any L stops that were outside of Chicago. Therefore, those rows were also dropped within `secondary_dataset_cleaning itself`. 

In [15]:
def remove_nan(total_pop):

    rows_with_nan = total_pop[total_pop.isna().any(axis = 1)]

    for index, row in rows_with_nan.iterrows():
        
        # If NaN value is for 2000 population
        if pd.isna(row['2000 Population']):
            row_population_2010 = row['2010 Population']
            row_population_2020 = row['2020 Population']

            # Used nominal change instead of percentage change
            nominal_change = row_population_2020 - row_population_2010

            # Calulate and update dataframe with extrapoplated value
            # Extrapolation uses subtraction because we are moving backwards in time
            extrapolate_row_population_2000 = int(row_population_2010 - nominal_change)
            
            if extrapolate_row_population_2000 < 1:
                total_pop.at[index, '2000 Population'] = 0
            
            else:
                total_pop.at[index, '2000 Population'] = extrapolate_row_population_2000

        # If NaN value is for 2020 population
        if pd.isna(row['2020 Population']):
            row_population_2000 = row['2000 Population']
            row_population_2010 = row['2010 Population']

            nominal_change = row_population_2010 - row_population_2000

            # Calulate and update dataframe with extrapoplated value
            # Extrapolation uses addition because we are moving forward in time
            extrapolate_row_population_2020 = int(row_population_2010 + nominal_change)
            
            if extrapolate_row_population_2020 < 1:
                total_pop.at[index, '2020 Population'] = 0
            
            else:
                total_pop.at[index, '2020 Population'] = extrapolate_row_population_2020
    
    return total_pop

In [16]:
df = secondary_dataset_cleaning()
df.head()

,ZIP,2000 Population,2010 Population,2020 Population
0,60004,20858.0,50582.0,52546.0
1,60005,13033.0,29308.0,30221.0
2,60007,13694.0,33820.0,33670.0
3,60008,8675.0,22717.0,22760.0
4,60010,14489.0,44095.0,46365.0


## Tertiary Dataset Cleaning

### Zip Code Identification and Collection using longitude and latitude

The third dataset we used contains information about each CTA L Stop. We want to use that information to be able to compare our ridership data with our population by ZIP Code data. The third dataset does not have the ZIP Code of each station, but it does have the longitude and latitude. Using the OpenCage's Geocoding API we were able to get the Zip Code for each L Stop by passing the API our latitude and longitude. The below functions format the data we have in a way that the Geocading API can read and retrieve the ZIP Code. 

In [17]:
# Zip Code Functions

def extract_lat_lon(location):
    location = location.strip('()')
    latitude, longitude = location.split(',')
    return float(latitude), float(longitude)

# Register following this url and get the API Key
# https://opencagedata.com/dashboard#geocoding


def get_zip_code(latitude, longitude):
    # read key and create geocoder
    # We originally have the key in Environment variable, but hard cording here for project report submission

    key = "606e648813dc4bbabeb0a392445c621f"
    geocoder = OpenCageGeocode(key)
    
    result = geocoder.reverse_geocode(latitude, longitude)
    if result and len(result):
        for postcode in result[0]['components']:
            return result[0]['components']['postcode']

def apply_get_zip_code(row):  
    return get_zip_code(row['latitude'], row['longitude'])

tertiary_dataset_cleaning takes dataset 3, removes all the columns we don't need, and then pulls the ZIP Code using the ZIP Code functions above. 

`Convert to code block to see what the raw dataframe looks like`

`dataset3 = pd.read_csv("../src/data/CTA_Stops_20240526.csv")`
`display(dataset3.head())`

### Tertiary Cleaning function including Zip Code functions defined above

The free version of OpenCage's Geocoding API has a rate limit of 2,500 pulls daily. Considering we have nearly 180 L Stops, we downloaded a CSV of our finished product in case the rate limit was breached (which happened many times). load_data() attempts to call the API, but if we breach the ratelimit will use our CSV as a backup. 

In [18]:
def tertiary_dataset_cleaning():

    try:
    
        cta_l_stop_df = pd.read_csv("../data/CTA_Stops_20240526.csv")

        # Get rid of all columns we don't want
        cta_l_stop_df = (cta_l_stop_df[['DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
        'STATION_DESCRIPTIVE_NAME', 'Location']])

        # Many stations allow individuals to travel in multiple directions. We only care about the station itself
        # therefore we are dropping Direction_ID and removing all the duplicate Station_Descriptive_Names
        
        cta_l_stop_df = cta_l_stop_df.drop_duplicates(subset=['STATION_DESCRIPTIVE_NAME'])
        cta_l_stop_df = cta_l_stop_df.drop(['DIRECTION_ID', 'STOP_NAME'], axis = 1)

        cta_l_stop_df['STATION_DESCRIPTIVE_NAME'] = (cta_l_stop_df['STATION_DESCRIPTIVE_NAME']
        .replace(r'\s+', '', regex=True)
        )

        cta_l_stop_df[['latitude', 'longitude']] = (cta_l_stop_df['Location']
        .apply(lambda x: pd.Series(extract_lat_lon(x)))
        )

        cta_l_stop_df['zip_code'] = cta_l_stop_df.apply(apply_get_zip_code, axis=1)

        cta_l_stop_df = cta_l_stop_df[['STATION_NAME', 'STATION_DESCRIPTIVE_NAME', 'zip_code', 'latitude', 'longitude']]

    except Exception as e:

        cta_l_stop_df = pd.read_csv('../data/tertiary_dataset_cleaning_output_061224.csv')    

    return cta_l_stop_df

In [19]:
df = tertiary_dataset_cleaning()
df.head()

,STATION_NAME,STATION_DESCRIPTIVE_NAME,zip_code,latitude,longitude
0,Cicero,Cicero(PinkLine),60804,41.851820,-87.745336
1,Central Park,CentralPark(PinkLine),60623,41.853839,-87.714842
2,Halsted,Halsted(GreenLine),60621,41.778943,-87.644244
3,Cumberland,Cumberland(BlueLine),60631,41.984246,-87.838028
4,Racine,Racine(BlueLine),60607,41.875920,-87.659458


`Convert to code block to see what the population dataframe looks like after cleaning`
`display(tertiaryy_dataset_cleaning().head())`

## Merging cleaned DataFrames together

Now that all our datasets have been cleaned, we created the functions merge_secondary_and_tertiary and final_merge to merge all three of our dataframes together

In [20]:
def merge_secondary_and_tertiary():
    secondary = secondary_dataset_cleaning()
    #If teritiary dataset fails on opencage api, provide the backup tertiary dataset cleaned csv 
    tertiary = tertiary_dataset_cleaning()

    population_station_df = pd.merge(tertiary, secondary, left_on = 'zip_code', right_on = 'ZIP')

    population_station_df = population_station_df[['zip_code', 'STATION_DESCRIPTIVE_NAME',
    '2000 Population', '2010 Population', '2020 Population', 'latitude', 'longitude']]

    return population_station_df

In [21]:
# Attempt to fix this code

def final_merge():

    primary = primary_dataset_cleaning()
    secondary_and_tertiary = merge_secondary_and_tertiary()

    # 142 unique names in ridership. Created a lookup csv with STATION_NAME mapping
    # import the mapping table
    station_name_mapping = pd.read_csv("../data/station_lookup_table.csv")

    station_name_mapping_selected = station_name_mapping[['RIDERSHIP_STATION_NAME', 'STATION_NAME']]

    # First merge between primary dataset, and lookup table. Named merged_ridership_station_df
    merged_ridership_station_df = (pd.merge(primary, station_name_mapping_selected,
    left_on='STATION_NAME', right_on='RIDERSHIP_STATION_NAME')
    )

    # Second merge between merged_ridership_station_df and tertiary dataset
    merged_ridership_cta_l_stop_df = (pd.merge(merged_ridership_station_df, secondary_and_tertiary,
    left_on='STATION_NAME_y', right_on='STATION_DESCRIPTIVE_NAME')
    )
    
    merged_ridership_cta_l_stop_df = merged_ridership_cta_l_stop_df[
        ['STATION_DESCRIPTIVE_NAME', 'MONTH_BEGINNING', 
    'AVG_WEEKDAY_RIDES', 'AVG_SATURDAY_RIDES', 'AVG_SUNDAY-HOLIDAY_RIDES', 'MONTHTOTAL',
    'zip_code', '2000 Population', '2010 Population', '2020 Population', 'latitude', 'longitude']
    ]

    return merged_ridership_cta_l_stop_df

## Final Output

The below function was created to create a singular dataframe that can then be imported as part of the overall ipynb file for other notebooks. Try & Except were used to prevent our functions from breaking in the case of any dependency issues that arise post submission

In [22]:
def load_finalmerge_data():
    try:
        # Try to call the API to get the data
        df = final_merge()
    except Exception as e:
        # If API Rate Limit is exceeded, load from CSV
        df = pd.read_csv('../data/final_merge_061124.csv')
    return df

In [ ]:
final_merge_df = load_finalmerge_data()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7862ac62-05ae-412f-9bc1-59d586e87203' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>